此文件包含了计算coinrate所需的一些函数

# This is new project

目的：对两段时间序列进行分析，给出The aggregated precursor coincidence rate和 The aggregated trigger coincidence rate；


读取文件-->操作文件

## 算法的处理

import matplotlib.pyplot as plt
import random
import string
import numpy as np
from scipy import stats

### 基本函数

这是需要用到的函数

In [ ]:
def indicator_f(x,int_v):
    '''这是示性函数，判断x是否属于其后的区间
    '''
    if x>=0 and x<=int_v:
        return 1
    else:
        return 0

In [ ]:
def heaviside_f(x):
    '''阶跃函数，判断是否大于0
    '''
    if x>0:
        return 1
    else:
        return 0

### 分析函数-数据中的每一个数代表在时间轴的位置

这是计算coincidence rate 的函数，参数为【时间间隔，时间延迟，序列A，序列B】

\begin{equation}r_{p}(\Delta T, \tau)=\frac{1}{N_{A}} \sum_{i=1}^{N_{A}} \Theta\left[\sum_{j=1}^{N_{B}} 1_{[0, \Delta T]}\left(\left(t_{i}^{A}-\tau\right)-t_{j}^{B}\right)\right]\end{equation}

In [ ]:
def pre_coin_r(time_,delay_,seq_a,seq_b):
    '''输入：巧合时间、时间延迟、序列a、序列b(时间轴)；\n输出a、b的pre rate
    '''
    rate_r=0
    count_a=0
    count_b=0
    rate_all=0
    for ta in seq_a:
        count_a+=1
        hx=0
        for tb in seq_b:
            hx+=indicator_f(ta-tb-delay_,time_)
            #print('ca is:',count_a,'x is:',ta-tb-delay_,'hx is:',hx)
        #hx=np.array([indicator_f(ta-tb-delay_,time_) for tb in seq_b]).sum() # 替代循环的功能
        rate_all+=heaviside_f(hx)
        #print('rate_all is :',rate_all)
    rate_r=rate_all/count_a
    #print('rate is:',rate_r,'rate_all is :',rate_all,'count_a is:',count_a)
    return rate_r

使用np.meshgrid()将两个一维数据生成二维数据网格，可以减少循环迭代带来的时间浪费

seqa和seqb转换为二维的网格

In [ ]:
def pre_cr_(time_,delay_,seq_a,seq_b):
    '''输入：巧合时间、时间延迟、序列a、序列b(时间轴)；\n输出a、b的pre rate
    '''
    rate_r=0
    x,y=np.meshgrid(seq_a,seq_b)  # 生成二维网格
    nx=pd.DataFrame(x-y-delay_)  # 二维网格相减等效于一次循环
    nnnz=nx[nx>=0][nx<=time_].count()  # 挑选剩余的非零数，等价于示性函数的运算
    rate_r=nnnz[nnnz>0].count()/len(seq_a)  # 等价于heaviside函数的运算，计算得出coin rate
    return rate_r

$$
r_{t}(\Delta T, \tau)=\frac{1}{N_{B}} \sum_{j=1}^{N_{B}} \Theta\left[\sum_{i=1}^{N_{A}} 1_{[0, \Delta T]}\left(\left(t_{i}^{A}-\tau\right)-t_{j}^{B}\right)\right]
$$

In [ ]:
def tri_coin_r(time_,delay_,seq_a,seq_b):
    '''输入：巧合时间、时间延迟、序列a、序列b(时间轴)；\n输出a、b的tri rate
    '''
    rate_r=0
    count_a=0
    count_b=0
    rate_all=0
    for tb in seq_b:
        count_b+=1
        hx=0
        for ta in seq_a:
            hx+=indicator_f(ta-tb-delay_,time_)
            #print('ca is:',count_a,'x is:',ta-tb-delay_,'hx is:',hx)
        rate_all+=heaviside_f(hx)
        #print('rate_all is :',rate_all)
    rate_r=rate_all/count_b
    #print('rate is:',rate_r,'rate_all is :',rate_all,'count_a is:',count_a)
    return rate_r

### 强巧合关联分析

使用np.meshgrid()将两个一维数据生成二维数据网格，可以减少循环迭代带来的时间浪费

seqa和seqb转换为二维的网格

$$r_{double}(\Delta T)=\frac{1}{N_{A}} \sum_{i=1}^{N_{A}} \Theta\left[\sum_{j=1}^{N_{B}} \left(1_{[0, \Delta T]}(t_{j-1}^{B}-t_{i}^{A})\times 1_{[0, \Delta T]}(t_{i}^{A}-t_{j}^{B})\right)\right]$$

In [ ]:
def coinrate_strong(time_,seq_a,seq_b):
    '''输入：巧合时间、序列a、序列b(时间轴)；\n输出a、b的pre rate\n两个示性函数的版本，添加了更强的关联
    '''
    # 时间轴上，后面的事件数值大，tB-tA>0 表示B事件在A事件之后，因此二维网格应该与公式的表达不一样
    rate_r=0
    x,y1=np.meshgrid(seq_a,seq_b[:-1])  # 生成二维网格，y1不包括最后一个元素，对应前面的示性函数
    x,y2=np.meshgrid(seq_a,seq_b[1:])  # 生成二维网格，y2不包括第一个元素，对应后面的示性函数

    nx1=pd.DataFrame(x-y1)  # 二维网格相减等效于一次循环，前面的示性函数网格
    nx2=pd.DataFrame(y2-x)  # 二维网格相减等效于一次循环，后面的示性函数网格

    filt=nx1[nx1>=0][nx1<=time_]*nx2[nx2>=0][nx2<=time_] # 计算相乘的示性函数网格
    
    nnnz=filt.count()  # 挑选剩余的非零数
    
    rate_r=nnnz[nnnz>0].count()/len(seq_a)  # 计算得出coin rate

    return rate_r

### 分析函数-时间窗口以数据为中心

In [ ]:
def pre_cr_center(time_,delay_,seq_a,seq_b):
    '''输入：巧合时间区间、时间延迟、序列a、序列b(时间轴)；\n输出a、b的pre rate
    '''
    rate_r=0
    x,y=np.meshgrid(seq_a,seq_b)  # 生成二维网格
    nx=pd.DataFrame(x-y-delay_)  # 二维网格相减等效于一次循环
    nnnz=nx[nx>=-time_/2][nx<=time_/2].count()  # 挑选剩余的非零数
    rate_r=nnnz[nnnz>0].count()/len(seq_a)  # 计算得出coin rate
    return rate_r

### 分析函数-数据中的每一个数代表与前一个数的时间间隔

In [ ]:
def pre_coin_r_inter(time_,delay_,seq_a,seq_b):
    '''输入：巧合时间、时间延迟、序列a、序列b(时间间隔)；\n输出a、b的pre rate
    '''
    rate_r=0
    count_a=0
    count_b=0
    rate_all=0
    time_all_a=0
    time_all_b=0
    for ta in seq_a:
        count_a+=1
        hx=0
        time_all_a+=ta
        time_all_b=0
        for tb in seq_b:
            time_all_b+=tb
            hx+=indicator_f(time_all_a-time_all_b-delay_,time_)
            #print('ca is:',count_a,'x is:',ta-tb-delay_,'hx is:',hx)
        rate_all+=heaviside_f(hx)
        #print('rate_all is :',rate_all)
    rate_r=rate_all/count_a
    #print('rate is:',rate_r,'rate_all is :',rate_all,'count_a is:',count_a)
    return rate_r

In [ ]:
def tri_coin_r_inter(time_,delay_,seq_a,seq_b):
    '''输入：巧合时间、时间延迟、序列a、序列b(时间间隔)；\n输出a、b的tri rate
    '''
    rate_r=0
    count_a=0
    count_b=0
    rate_all=0
    time_all_a=0
    time_all_b=0
    for tb in seq_b:
        count_b+=1
        hx=0
        time_all_b+=tb
        time_all_a=0
        for ta in seq_a:
            time_all_a+=ta
            hx+=indicator_f(time_all_a-time_all_b-delay_,time_)
            #print('ca is:',count_a,'x is:',ta-tb-delay_,'hx is:',hx)
        rate_all+=heaviside_f(hx)
        #print('rate_all is :',rate_all)
    rate_r=rate_all/count_b
    #print('rate is:',rate_r,'rate_all is :',rate_all,'count_a is:',count_a)
    return rate_r

## 处理数据

### 数据的转换

#### 函数定义-相对前一时刻的时间间隔转换为在时间轴上的时刻

输入一个相对时间的序列，输出时间轴上的序列

In [ ]:
def transfrom_ts_to_t(ts):
    '''时间间隔转换为时间轴
    '''
    all_time=0
    t=[]
    for time in ts:
        all_time+=time
        t.append(all_time)
    return t

#### 函数定义，将二维数据写入文件中，参数为文件名、二维列表

In [ ]:
def write_two_l(doc_n,two_li):
    '''二维数据写入文件，输入：文件名、二维数据
    '''
    with open(doc_n,'w') as f:
        for y in range(0,len(two_li)):
            for x in range(0,len(two_li[0])):
                f.write('%.5f\t'%two_li[y][x])
            f.write('\n')

### 计算结果并保存到文件

生成基于两个序列的coin rate的数据，参数为B事件的数目以及时间间隔

#### 随机分布的结果

rp=0
with open('result001.txt','w') as f:
    for nb in range(20,0,-1):
        for time in range(1,21):
            rp=pre_coin_r(10*time,0,s1,s2[:nb*10])
            f.write('%.2f'%rp+'\t')
        f.write('\n')


rp=0
with open('result002.txt','w') as f:
    for nb in range(20,0,-1):
        for time in range(1,21):
            rp=tri_coin_r(10*time,0,s1,s2[:nb*10])
            f.write('%.2f'%rp+'\t')
        f.write('\n')


#### 泊松分布的结果

##### 试验

rp=0
s1=20*stats.poisson.rvs(mu=2,size=100)
s2=20*stats.poisson.rvs(mu=2,size=200)
print(len(s1),len(s2))
print('mean:%g , sd:%g '%(np.mean(s1),np.std(s1,ddof=1)))
plt.hist(s1,bins=10)
plt.show()
with open('result_poisson_001.txt','w') as f:
    for nb in range(20,0,-1):
        for time in range(1,21):
            rp=pre_coin_r(10*time,0,s1,s2[:nb*10])
            f.write('%.2f'%rp+'\t')
        f.write('\n')


不同的参数对coin rate的影响，泊松分布的均值是参数mu

In [ ]:
def dif_para_rp(doc_n):
    with open(doc_n,'w') as f:
        for para_mu in np.arange(1,20,1):
            arp=[]
            print('this is para %.1f'%para_mu)
            for i in range(0,100):
                seq_a=transfrom_ts_to_t(stats.poisson.rvs(mu=para_mu,loc=0,size=10))
                seq_b=transfrom_ts_to_t(stats.poisson.rvs(mu=para_mu,loc=0,size=20))
                seq_b=[x/2 for x in seq_b]
                time_a=100*para_mu
                rate_p=[]
                print(i)
#                print(time_a,int(time_a*0.2)//20)
                for td in range(0,20):
                    rate_p.append(pre_coin_r(td*0.01*time_a,0,seq_a,seq_b))
                arp.append(rate_p)
            for x1 in range(0,len(arp[0])):
                temp_=0
                for x2 in range(0,len(arp)):
                    temp_+=arp[x2][x1]
#                print(type(temp_),type(len(arp)))
                f.write('%.5f\t'%(temp_/len(arp)))
            f.write('\n')

dif_para_rp('text_dif2.txt')

##### 定义函数-生成不同的两个随机序列计算coin rate 并且将结果存入文件，参数为文件名、泊松分布的参数mu，事件数目

In [ ]:
def dif_nb_rp(doc_n,para_mu,size_a):
    with open(doc_n,'w') as f:
        seq_a=transfrom_ts_to_t(stats.poisson.rvs(mu=para_mu,loc=0,size=size_a))
        seq_b=transfrom_ts_to_t(stats.poisson.rvs(mu=para_mu/2,loc=0,size=size_a))
        time_a=10*para_mu
        for nb in range(1,21):
            nsb=random.sample(seq_b,nb*size_a*2//20)
            print(nsb,seq_a)
            for ti in range(0,20):
                rate_p=pre_coin_r(ti*0.01*time_a,0,seq_a,nsb)
                f.write('%.5f\t'%rate_p)
            f.write('\n')

##### 定义函数-生成不同的两个随机序列计算coin rate 并且将结果返回二维列表，参数泊松分布的参数mu，事件的数目

给出的二维数据的横坐标是时间间隔，纵坐标是B事件数目

In [ ]:
def dif_nb_rp_list(para_mu,size_a):
    '''输入：分布参数，事件数量；\n输出：二维列表
    '''
    seq_a=transfrom_ts_to_t(stats.poisson.rvs(mu=para_mu,loc=0,size=size_a))
    seq_b=transfrom_ts_to_t(stats.poisson.rvs(mu=para_mu/2,loc=0,size=2*size_a))
    time_a=size_a*para_mu
    lis0=[]
    for nb in range(1,21):
        nsb=random.sample(seq_b,nb*size_a*2//20)
        #print(nsb,seq_a)
        list1=[]
        for ti in range(0,20):
            rate_p=pre_coin_r(ti*0.01*time_a,0,seq_a,nsb)
            list1.append(rate_p)
        lis0.append(list1)
    return lis0[::-1]

##### 计算多组数据并且取平均后返回一个二维列表

###### 计算均值和标准差

对大量的二维数据求平均以及标准差

f=[]
for i in range(20):
    f.append(dif_nb_rp_list(20,100))


write_two_l('0000001.txt',np.mean(f,axis=0))

write_two_l('0000001.txt',np.std(f,axis=0))

###### 计算均值

用三维列表记录20组coin rate 的结果，每一维都是一组以x为时间间隔、y为b时间数目的数据

fn_a=[]
for fn in range(0,100):
    fn_a.append(dif_nb_rp_list(10,10))

print(len(fn_a),len(fn_a[0]),len(fn_a[0][0]),fn_a[0][0][0])

list_y=[]
for x in range(0,20):
    print('x=',x)
    list_x=[]
    for y in range(0,20):
        print('y=',y)
        list_nn=0
        for fnn in fn_a:
            list_nn+=fnn[x][y]
        print(list_nn)
        list_x.append(list_nn/len(fn_a))
        print(list_x)
    list_y.append(list_x)

print(list_y,len(list_y),len(list_y[0]))

write_two_l('possion_mean20_rp2.txt',list_y)

###### 计算标准差

fn_a=[]
for fn in range(0,20):
    fn_a.append(dif_nb_rp_list('dif_para_pl_rp_test.txt',10))

print(len(fn_a),len(fn_a[0]),len(fn_a[0][0]),fn_a[0][0][0])

list_y=[]
for x in range(0,20):
    print('x=',x)
    list_x=[]
    for y in range(0,20):
        print('y=',y)
        list_nn=[]
        for fnn in fn_a:
            list_nn.append(fnn[x][y])
        print(list_nn)
        list_x.append(np.std(np.array(list_nn)))
        print(list_x)
    list_y.append(list_x)

print(list_y,len(list_y),len(list_y[0]))

write_two_l('possion_std20_rp.txt',list_y)

#### 幂律分布的结果

##### 试验

with open('power_rp_.txt','w') as f:
    for i in range(0,100):
        seq_a=transfrom_ts_to_t(stats.powerlaw.rvs(2,size=10))
        seq_b=transfrom_ts_to_t(stats.powerlaw.rvs(2,size=10))
        #print(seq_a,'/n',seq_b)
        for td in range(0,20):
            rate_p=pre_coin_r(td,0,seq_a,seq_b)
            f.write('%.5f\t'%rate_p)
        #print(seq_a)
        f.write('\n')

幂律分布的均值是$$\frac{a}{a+1}$$,a是参数

这个函数的参数是：文件名、事件的数目、每个数据需要的实验数，输出文件以时间间隔为x轴，分布函数参数为y轴

In [ ]:
def dif_para_pl_rp(doc_n,size_pot,times_test):
    with open(doc_n,'w') as f:
        for para_a in range(1,21):
            arp=[]
            print('this is para %d'%para_a)
            for i in range(0,times_test):
                seq_a=transfrom_ts_to_t(stats.powerlaw.rvs(para_a,size=size_pot))
                seq_b=transfrom_ts_to_t(stats.powerlaw.rvs(para_a,size=2*size_pot))
                seq_b=[x/2 for x in seq_b]
                time_a=10*para_a/(para_a+1)
                rate_p=[]
                print(i)
#                print(time_a,int(time_a*0.2)//20)
                for td in range(0,20):
                    rate_p.append(pre_coin_r(td*0.01*time_a,0,seq_a,seq_b))
                arp.append(rate_p)
            for x1 in range(0,len(arp[0])):
                temp_=0
                for x2 in range(0,len(arp)):
                    temp_+=arp[x2][x1]
#                print(type(temp_),type(len(arp)))
                f.write('%.5f\t'%(temp_/len(arp)))
            f.write('\n')

dif_para_pl_rp('dif_para_pl_rp_test.txt',100,20)

##### 定义函数-生成不同的两个随机序列计算coin rate 并且将结果返回二维列表，参数为幂律分布的参数a，事件数量

In [ ]:
def dif_nb_pol_rp_list(para_a,size_a):
    '''输入：分布参数，事件数量；\n输出：二维列表
    '''
    seq_a=transfrom_ts_to_t(stats.powerlaw.rvs(para_a,size=size_a))
    seq_b=[i/2 for i in transfrom_ts_to_t(stats.powerlaw.rvs(para_a,size=2*size_a))]
    time_a=size_a*para_a/(para_a+1)
    lis0=[]
    for nb in range(1,21):
        nsb=random.sample(seq_b,nb*size_a*2//20)
        #print(nsb,seq_a)
        list1=[]
        for ti in range(0,20):
            rate_p=pre_coin_r(ti*0.01*time_a,0,seq_a,nsb)
            list1.append(rate_p)
        lis0.append(list1)
    return lis0[::-1]

比较耗费时间的：

fn_a=[]
for fn in range(0,20):
    fn_a.append(dif_nb_pol_rp_list(5,10))

print(len(fn_a),len(fn_a[0]),len(fn_a[0][0]),fn_a[0][0][0])

list_y=[]
for x in range(0,20):
    print('x=',x)
    list_x=[]
    for y in range(0,20):
        print('y=',y)
        list_nn=0
        for fnn in fn_a:
            list_nn+=fnn[x][y]
        print(list_nn)
        list_x.append(list_nn/len(fn_a))
        print(list_x)
    list_y.append(list_x)

print(list_y,len(list_y),len(list_y[0]))

s=dif_nb_pol_rp_list(5,10)
print(s,type(s),len(s[0]))

write_two_l('pol_mean_nb_test.txt',list_y)

#fn_a=[]
#for fn in range(0,20):
#fn_a.append(dif_nb_rp_list('dif_para_pl_rp_test.txt',10))

#print(len(fn_a),len(fn_a[0]),len(fn_a[0][0]),fn_a[0][0][0])

list_y=[]
for x in range(0,20):
    print('x=',x)
    list_x=[]
    for y in range(0,20):
        print('y=',y)
        list_nn=[]
        for fnn in fn_a:
            list_nn.append(fnn[x][y])
        print(list_nn)
        list_x.append(np.std(np.array(list_nn)))
        print(list_x)
    list_y.append(list_x)

print(list_y,len(list_y),len(list_y[0]))

write_two_l('pol_std_nb_test.txt',list_y)

##### 计算均值和标准差

对大量的二维数据求平均以及标准差 

a=[]
for i in range(20):
    a.append(dif_nb_pol_rp_list(5,10))


write_two_l('powlaw_.txt',np.mean(a,axis=0))

write_two_l('powlaw_.txt',np.std(a,axis=0))

#### 幂律-泊松分布的coin rate

In [ ]:
def dif_nb_pow_isson(para_a,para_b,size_a):
    '''输入：幂律分布参数，泊松分布参数，事件数量；\n输出：二维列表
    '''
    size_b=2*size_a
    seq_a=transfrom_ts_to_t(stats.powerlaw.rvs(para_a,size=size_a))
    temp=transfrom_ts_to_t(stats.poisson.rvs(para_b,size=size_b))
    
    time_a=size_a*para_a/(para_a+1)
    time_b=size_b*para_b
    
    seq_b=[i*(time_a/time_b) for i in temp]

    lis0=[]
    for nb in range(1,21):
        nsb=random.sample(seq_b,nb*size_b//20)
        #print(nsb,seq_a)
        list1=[]
        for ti in range(0,20):
            rate_p=pre_coin_r(ti*0.01*time_a,0,seq_a,nsb)
            list1.append(rate_p)
        lis0.append(list1)
    return lis0[::-1]

#### 计算任意两个序列的

In [ ]:
def dif_nb_seq(seq_a,seq_b):
    '''输入：事件序列a，事件序列b；\n输出：二维列表
    '''
    size_a=len(seq_a)
    size_b=len(seq_b)

    time_a=seq_a[-1]
    time_b=seq_b[-1]
    
    seq_b=[i*(time_a/time_b) for i in seq_b]

    lis0=[]
    for nb in range(1,21):
        nsb=random.sample(seq_b,nb*size_b//20)
        #print(nsb,seq_a)
        list1=[]
        for ti in range(0,20):
            rate_p=pre_coin_r(ti*0.01*time_a,0,seq_a,nsb)
            list1.append(rate_p)
        lis0.append(list1)
    return lis0[::-1]

In [ ]:
def dif_seq_inter(seq_a,seq_b,inter=0,d_t=0):
    '''输入：事件序列a，事件序列b，联系强度，联系延迟；\n输出：二维列表\n这个作用直接添加到b序列
    '''
    size_a=len(seq_a)
    size_b=len(seq_b)
    
    seq_a=np.array(seq_a)
    seq_b=np.array(seq_b)

    time_a=seq_a[-1]
    time_b=seq_b[-1]
    
    #seq_b=[i*(time_a/time_b) for i in seq_b]
    seq_b=(time_a/time_b)*seq_b
    
    seq_b=np.append(seq_b,np.random.choice(seq_a,int(inter*size_a),replace=False)-d_t*time_a)

    lis0=[]
    for nb in range(1,21):
        nsb=np.random.choice(seq_b,nb*size_b//20,replace=False)
        #print(nsb,seq_a)
        list1=[]
        for ti in range(0,20):
            rate_p=pre_coin_r(ti*0.01*time_a,0,seq_a,nsb)
            list1.append(rate_p)
        lis0.append(list1)
    return lis0[::-1]

In [ ]:
def dif_seq_inter2(seq_a,seq_b,inter=0,d_t=0):
    '''输入：事件序列a，事件序列b，联系强度，联系延迟；\n输出：二维列表\n这个作用同时也包括了b序列的改变
    '''
    size_a=len(seq_a)
    size_b=len(seq_b)
    
    seq_a=np.array(seq_a)
    seq_b=np.array(seq_b)

    time_a=seq_a[-1]
    time_b=seq_b[-1]
    
    #seq_b=[i*(time_a/time_b) for i in seq_b]
    seq_b=(time_a/time_b)*seq_b
    
    seq_b=np.append(np.random.choice(seq_b,int(size_b)-int(inter*size_a),replace=False),
                    np.random.choice(seq_a,int(inter*size_a),replace=False)-d_t*time_a)

    lis0=[]
    for nb in range(1,21):
        nsb=np.random.choice(seq_b,nb*size_b//20,replace=False)
        #print(nsb,seq_a)
        list1=[]
        for ti in range(0,20):
            rate_p=pre_coin_r(ti*0.01*time_a,0,seq_a,nsb)
            list1.append(rate_p)
        lis0.append(list1)
    return lis0[::-1]

In [ ]:
def dif_seq_inter2n(seq_a,seq_b,inter=0,d_t=0):
    '''输入：事件序列a，事件序列b，联系强度，联系延迟；\n输出：二维列表\n这个作用同时也包括了b序列的改变
    '''
    size_a=len(seq_a)
    size_b=len(seq_b)
    
    seq_a=np.array(seq_a)
    seq_b=np.array(seq_b)

    time_a=seq_a[-1]
    time_b=seq_b[-1]
    
    #seq_b=[i*(time_a/time_b) for i in seq_b]
    seq_b=(time_a/time_b)*seq_b
    
    seq_b=np.append(np.random.choice(seq_b,int(size_b)-int(inter*size_a),replace=False),
                    np.random.choice(seq_a,int(inter*size_a),replace=False)-d_t*time_a)

    lis0=[]
    for nb in range(1,21):
        nsb=np.random.choice(seq_b,nb*size_b//20,replace=False)
        #print(nsb,seq_a)
        list1=[]
        for ti in range(0,20):
            rate_p=pre_cr_(ti*0.01*time_a,0,seq_a,nsb)
            list1.append(rate_p)
        lis0.append(list1)
    return lis0[::-1]

a=[]
for i in range(20):
    a.append(dif_nb_pow_isson(5,5,10))

write_two_l('pow_isson.txt',np.mean(a,axis=0))

write_two_l('pow_isson.txt',np.std(a,axis=0))

#### 两个事件的coinrate strong

In [ ]:
def disq_cr_strong(seq_a,seq_b,delta_t=600,inter=0,d_t=0):
    '''输入：事件序列a，事件序列b，时间窗口=600；\n输出：二维列表\n这个作用直接添加到b序列
    '''
    size_a=len(seq_a)
    size_b=len(seq_b)
    
    seq_a=np.array(seq_a)
    seq_b=np.array(seq_b)

    time_a=seq_a[-1]
    time_b=seq_b[-1]
    
    lis0=[]
    for nb in range(1,21):
        nsb=np.random.choice(seq_b,nb*size_b//20,replace=False)
        list1=[]
        for na in range(1,21):
            nsa=np.random.choice(seq_a,na*size_a//20,replace=False)
            rate_p=coinrate_strong(delta_t,nsa,nsb)
            list1.append(rate_p)
            print('successed ',na,nb)
        lis0.append(list1)
    
    return lis0[::-1]

#### intertime coinrate strong

In [ ]:
def disq_cr_strong_inter(seq_a,seq_b,delta_t=600,):
    '''输入：事件序列a，事件序列b，时间窗口=600；\n输出：x、y、数据的二维列表
    '''
    size_a=len(seq_a)
    size_b=len(seq_b)

    seq_a=np.array(seq_a)
    seq_b=np.array(seq_b)
    
    y=[]
    lis0=[]
    for nb in range(1,21):
        tsb=nb*size_b//20
        y.append(tsb)
        nsb=np.random.choice(seq_b,tsb,replace=False)
        nsb=transfrom_ts_to_t(nsb)
        
        x=[]
        list1=[]
        for na in range(1,21):
            tsa=na*size_a//20
            x.append(tsa)
            nsa=np.random.choice(seq_a,tsa,replace=False)
            nsa=transfrom_ts_to_t(nsa)
            
            rate_p=coinrate_strong(delta_t,nsa,nsb)
            list1.append(rate_p)
        lis0.append(list1)
    
    return x,y,lis0

## 计算成功

In [ ]:
print("coin rate calculate succeed!")